In [5]:
print("hello")

hello


In [7]:
import pandas as pd

# Step 1: Load the check-in data
df = pd.read_json('yelp_academic_dataset_checkin.json', lines=True)

In [8]:
print(df.head())

              business_id                                               date
0  ---kPU91CF4Lq2-WlRu9Lw  2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020...
1  --0iUa4sNDFiZFrAdIWhZQ  2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011...
2  --30_8IhuyMHbSOcNWd6DQ           2013-06-14 23:29:17, 2014-08-13 23:20:22
3  --7PUidqRWpRSpXebiyxTg  2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012...
4  --7jw19RH9JKXgFohspgQw  2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014...


In [10]:
# Step 2: Parse and explode timestamp lists
df['date'] = df['date'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])
df_exploded = df.explode('date')
df_exploded['date'] = pd.to_datetime(df_exploded['date'], errors='coerce')

# Step 3: Extract day of week
df_exploded['day_of_week'] = df_exploded['date'].dt.day_name()

# Step 4: Aggregate to compact format
compact_df = (
    df_exploded.groupby('business_id')
    .agg(
        total_checkins=('date', 'count'),
        first_checkin=('date', 'min'),
        last_checkin=('date', 'max')
    )
    .reset_index()
)

# Step 5: Get day-wise check-in counts
checkins_by_day = (
    df_exploded.groupby(['business_id', 'day_of_week'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# Step 6: Merge day-of-week counts into the compact summary
compact_df = pd.merge(compact_df, checkins_by_day, on='business_id', how='left')

# Optional: Fill missing weekday columns with 0
weekday_columns = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for day in weekday_columns:
    if day not in compact_df.columns:
        compact_df[day] = 0



In [12]:
print(compact_df.head())

              business_id  total_checkins       first_checkin  \
0  ---kPU91CF4Lq2-WlRu9Lw              11 2020-03-13 21:10:56   
1  --0iUa4sNDFiZFrAdIWhZQ              10 2010-09-13 21:43:09   
2  --30_8IhuyMHbSOcNWd6DQ               2 2013-06-14 23:29:17   
3  --7PUidqRWpRSpXebiyxTg              10 2011-02-15 17:12:00   
4  --7jw19RH9JKXgFohspgQw              26 2014-04-21 20:42:11   

         last_checkin  Friday  Monday  Saturday  Sunday  Thursday  Tuesday  \
0 2021-11-11 16:23:50       4       0         2       0         1        2   
1 2014-04-12 23:04:47       1       4         1       0         0        2   
2 2014-08-13 23:20:22       1       0         0       0         0        0   
3 2015-09-27 13:18:32       1       1         1       2         2        3   
4 2021-06-21 19:59:50       0       9         0       1         9        5   

   Wednesday  
0          2  
1          2  
2          1  
3          0  
4          2  


In [14]:
from IPython.display import FileLink

# Save the cleaned DataFrame to a JSON file in the current working directory
compact_df.to_csv('yelp_business_checkins_transformed.csv')
print("Hello")
# Create a download link for the JSON file
FileLink('yelp_business_checkins_transformed.csv')

Hello


/Users/abhishekkumar/Downloads/Yelp JSON/yelp_business_checkins_transformed.csv